<a href="https://colab.research.google.com/github/Krishnan-Raghavan/Packt/blob/main/GameTheoryChapter7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# play_tictactoe.py
from tictactoe_board import TicTacToeBoard
from tictactoe_types import Player, Move
from tictactoe_ai import TicTacToeAI

def main():
    """
    Main function to run the Tic Tac Toe game.
    """
    board = TicTacToeBoard()
    player = Player.X
    ai = TicTacToeAI(player)

    while not board.is_full() and not board.is_winner(Player.X) and not board.is_winner(Player.O):
        print_board(board)
        if player == Player.X:
            move = ai.best_move(board)
        else:
            move = get_human_move(board)
        board.make_move(move, player)
        player = player.opponent()

    print_board(board)
    if board.is_winner(Player.X):
        print("Player X wins!")
    elif board.is_winner(Player.O):
        print("Player O wins!")
    else:
        print("It's a tie!")

def get_human_move(board):
    """
    Prompts the human player to enter a move and validates the input.
    """
    while True:
        try:
            move = int(input("Enter your move (0-8): "))
            if move in [m.position for m in board.available_moves()]:
                return Move(move)
            else:
                print("Invalid move. Try again.")
        except ValueError:
            print("Invalid input. Enter a number between 0 and 8.")

def print_board(board):
    """
    Prints the current state of the board.
    """
    for i in range(0, 9, 3):
        row = [board.board[i + j].value if board.board[i + j] is not None else '-' for j in range(3)]
        print(' '.join(row))
    print()

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'tictactoe_board'

In [ ]:
# tictactoe_ai.py
from tictactoe_board import TicTacToeBoard
from tictactoe_types import Move, Player

class TicTacToeAI:
    """
    Minimax algorithm implementation for Tic Tac Toe.
    """
    def __init__(self, player: Player):
        self.player = player
        self.evaluated_states = 0

    def best_move(self, board: TicTacToeBoard) -> Move:
        """
        Determines the best move for the current player using the Minimax algorithm.
        """
        best_value = -float('inf')
        best_move = None
        for move in board.available_moves():
            board.make_move(move, self.player)
            value = self._evaluate(board, False)
            board.undo_move(move)
            if value > best_value:
                best_value = value
                best_move = move
        return best_move

    def _evaluate(self, board: TicTacToeBoard, is_maximizing: bool) -> float:
        """
        Recursively evaluates the board states to determine the best move.
        """
        self.evaluated_states += 1  # Increment the counter
        if board.is_winner(self.player):
            return 1
        elif board.is_winner(self.player.opponent()):
            return -1
        elif board.is_full():
            return 0

        if is_maximizing:
            best_value = -float('inf')
            for move in board.available_moves():
                board.make_move(move, self.player)
                value = self._evaluate(board, False)
                board.undo_move(move)
                best_value = max(best_value, value)
            return best_value
        else:
            best_value = float('inf')
            for move in board.available_moves():
                board.make_move(move, self.player.opponent())
                value = self._evaluate(board, True)
                board.undo_move(move)
                best_value = min(best_value, value)
            return best_value

In [ ]:
# tictactoe_board.py
from tictactoe_types import Move, Player

class TicTacToeBoard:
    """
    Represents the Tic Tac Toe board and handles the game state.
    """
    def __init__(self):
        self.board = [None] * 9

    def make_move(self, move: Move, player: Player):
        """
        Places the player's mark on the board at the specified position.
        """
        if self.board[move.position] is None:
            self.board[move.position] = player

    def undo_move(self, move: Move):
        """
        Removes the player's mark from the board at the specified position.
        """
        self.board[move.position] = None

    def available_moves(self):
        """
        Returns a list of available moves on the board.
        """
        return [Move(i) for i, x in enumerate(self.board) if x is None]

    def is_winner(self, player: Player) -> bool:
        """
        Checks if the specified player has won the game.
        """
        winning_combinations = [
            (0, 1, 2), (3, 4, 5), (6, 7, 8),  # rows
            (0, 3, 6), (1, 4, 7), (2, 5, 8),  # columns
            (0, 4, 8), (2, 4, 6)  # diagonals
        ]
        return any(all(self.board[i] == player for i in combo) for combo in winning_combinations)

    def is_full(self) -> bool:
        """
        Checks if the board is full.
        """
        return all(x is not None for x in self.board)

In [ ]:
# tictactoe_types.py
from enum import Enum

class Player(Enum):
    """
    Represents a player in the Tic Tac Toe game.
    """
    X = 'X'
    O = 'O'

    def opponent(self):
        """
        Returns the opponent of the current player.
        """
        return Player.X if self == Player.O else Player.O

class Move:
    """
    Represents a move in the Tic Tac Toe game.
    """
    def __init__(self, position: int):
        self.position = position